In [9]:
# 导入 joblib 模块
from joblib import dump

print("\n保存最佳CS-Stacking模型...")
dump(best_model, 'CS-Stacking-model.joblib')
print("模型已成功保存为 CS-Stacking-model.joblib")


保存最佳CS-Stacking模型...
模型已成功保存为 CS-Stacking-model.joblib


In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox, filedialog
import numpy as np
from joblib import load
import warnings

class featureNameSetter:
    def __init__(self, model=None, feature_names=None):
        self.model = model
        self.feature_names = feature_names

    def set_feature_names(self, feature_names):
        self.feature_names = feature_names

# 初始化主窗口
predict_root = tk.Tk()
predict_root.title("Optimization Prediction Interface")
predict_root.geometry("1600x800")

warnings.filterwarnings('ignore', category=UserWarning)

# 设置全局样式
style = ttk.Style(predict_root)
available_themes = style.theme_names()
print("可用主题:", available_themes)  

try:
    style.theme_use('classic')  # 或 'alt', 'classic'
except:
    print("clam主题不可用，使用默认主题")

# 配置样式
style.configure("TEntry", 
                font=('Times New Roman', 20, "bold"),  
                foreground='black',
                padding=5)

style.configure("TFrame", borderwidth=5, relief="groove")
style.configure("TLabel", font=('Times New Roman', 16, "bold"))
style.configure("TButton",
                background='#0078D4',
                foreground='black',
                bordercolor='#0078D4',
                borderwidth=1,
                font=('Times New Roman', 18, "bold"))
style.configure("TCombobox",
                font=('Times New Roman', 16, 'bold'),
                foreground='black') 
style.map('TButton',
          background=[('active', '#005A9E'), ('pressed', '#004A8C')],
          foreground=[('active', 'white'), ('pressed', 'white')])

# 如果ttk.Entry样式不生效，使用这个函数创建自定义Entry
def create_styled_entry(parent, textvariable=None, width=12):
    """创建具有自定义样式的输入框"""
    return tk.Entry(parent, 
                   textvariable=textvariable, 
                   width=width,
                   font=('Times New Roman', 16, 'bold'),  # 直接设置字体
                   bg='white', 
                   fg='black',
                   relief='solid',
                   bd=1)

# 定义目标函数
def aim_function(**kwargs):
    try:
        try:
            models = {
                'CS': load('CS-Stacking-model.joblib'),
            }
        except Exception as e:
            messagebox.showerror("模型加载错误", f"无法加载模型文件: {str(e)}\n请确保模型文件 'CS-Stacking-model.joblib' 存在。")
            return [0.0]
        
        # 调整后的特征映射
        feature_mapping = {
            'CS': ['MPD', 'Si/Al', 'W/B', 'AC', 'SM', 'CA', 'IT', 'TT', 'VD'],
        }
        
        # 验证模型是否正确加载
        if not hasattr(models['CS'], 'predict'):
            raise ValueError("模型文件加载失败，请检查模型文件是否正确。")
        
        # 验证特征映射是否正确
        if len(feature_mapping['CS']) != 9:
            raise ValueError("特征映射长度不正确，请检查特征映射是否与模型训练时一致。")
        
        # 预测CS
        cs_features = [kwargs[var] for var in feature_mapping['CS']]
        
        # 验证输入特征是否在合理范围内
        if not (0 <= kwargs['MPD'] <= 247.45 and 
                0.98 <= kwargs['Si/Al'] <= 5.61 and 
                0.04 <= kwargs['W/B'] <= 0.491 and 
                0 <= kwargs['AC'] <= 0.155 and 
                0 <= kwargs['SM'] <= 3.34 and 
                0 <= kwargs['CA'] <= 108 and 
                -196 <= kwargs['IT'] <= 120 and 
                -196 <= kwargs['TT'] <= 600 and 
                kwargs['VD'] in [10, 100, 101325]): 
            raise ValueError("输入特征超出合理范围，请检查输入数据。")
        
        # 预测结果
        cs = models['CS'].predict(np.array(cs_features).reshape(1, -1))[0]
        
        return [cs]
    
    except Exception as e:
        messagebox.showerror("Error", f"模型预测错误: {str(e)}")
        return [0.0]

# 定义更新函数
def update_predict():
    try:
        input_values = {}
        for var_name, var in input_vars.items():
            if var_name == 'AC':
                ac_value = var.get()
                try:
                    ac_value = float(ac_value) / 100
                except ValueError:
                    ac_value = float(ac_value)
                input_values[var_name] = ac_value
            else:
                input_values[var_name] = float(var.get())
        
        # 预测结果
        results = aim_function(**input_values)
        
        # 更新输出
        for i, (result_var, result) in enumerate(zip(output_vars, results)):
            formatted_result = f"{result:.2f}"
            result_var.delete(0, tk.END)
            result_var.insert(0, formatted_result)
            
    except ValueError as e:
        messagebox.showerror("Error", f"输入数据格式错误: {str(e)}")
    except Exception as e:
        messagebox.showerror("Error", f"预测过程中发生错误: {str(e)}")

def clear_entries():
    for var_name, var in input_vars.items():
        if var_name == 'VD':
            var.set("101325")  # VD恢复默认值
        else:
            var.set("")
    for entry in output_vars:
        entry.delete(0, tk.END)

def save_entries():
    file_path = filedialog.asksaveasfilename(defaultextension=".txt", filetypes=[("Text Files", "*.txt"), ("All Files", "*.*")])
    if file_path:
        with open(file_path, "w") as file:
            input_data = ""
            for var_name, var in input_vars.items():
                input_data += f"{var_name}{input_units[var_name]}: {var.get()}\n"
            
            output_data = ""
            for i, entry in enumerate(output_vars):
                output_data += f"{output_labels[i]}{output_units[output_labels[i]]}: {entry.get()}\n"
            
            file.write(input_data + "\n" + output_data)
            messagebox.showinfo("保存成功", "数据已成功保存到文件！")

def return_main_menu():
    predict_root.destroy()

input_units = {
    'MPD': ' (µm)',
    'Si/Al': ' (--)',
    'W/B': ' (--)',
    'AC': ' (%)',
    'SM': ' (--)',
    'CA': ' (days)',
    'IT': ' (°C)',
    'TT': ' (°C)',
    'VD': ' (Pa)'
}

output_units = {
    'CS': ' (MPa)'
}

title_label = tk.Label(predict_root, text="Optimization Prediction Interface", font=('Times New Roman', 24, "bold"))
title_label.grid(row=0, column=0, columnspan=3, sticky="ew", padx=10, pady=10)

input_outer_frame = ttk.Frame(predict_root, padding="0 0 0 0", style="TFrame")
input_outer_frame.grid(row=1, column=0, sticky="nsew", padx=(5, 5))

input_title_label = ttk.Label(input_outer_frame, text="Input panel", font=('Times New Roman', 18, "bold"))
input_title_label.pack(fill="x", padx=300, pady=10)

input_frame = ttk.Frame(input_outer_frame, padding="6 6 6 15")
input_frame.pack(fill="both", expand=True, padx=6, pady=5)

input_vars = {}
original_input_labels = ['MPD', 'Si/Al', 'W/B', 'AC', 'SM', 'CA', 'IT', 'TT', 'VD']
input_labels = ['MPD', 'Si/Al', 'W/B', 'AC', 'SM', 'CA', 'IT', 'TT', 'VD']

default_values = {
    'MPD': '38.20',
    'Si/Al': '2.32',
    'W/B': '0.30',
    'AC': '10.00',
    'SM': '0',
    'CA': '3.00',
    'IT': '93.00',
    'TT': '99.60',
    'VD': '101325'
}

# 创建输入控件（三列，每列3行）
for i, label in enumerate(input_labels):
    row, col = divmod(i, 3)
    # 添加单位
    full_label = f"{label}{input_units[label]}"
    ttk.Label(input_frame, text=full_label).grid(row=row, column=col*2, padx=7, pady=15, sticky="ew")
    input_vars[label] = tk.StringVar(value=default_values.get(label, ""))
    
    if label == 'VD':
        # VD使用下拉选择框，限制只能选10、100、101325
        vd_combobox = ttk.Combobox(
            input_frame,
            textvariable=input_vars[label],
            values=(10, 100, 101325),  # 限制可选值
            width=13,
            font=('Times New Roman', 16, 'bold'),
            state='readonly'  # 只读模式，禁止手动输入
        )
        vd_combobox.grid(row=row, column=col*2+1, padx=8, pady=30, sticky="ew")
    else:
        # 其他参数使用普通输入框
        entry = create_styled_entry(input_frame, textvariable=input_vars[label], width=13)
        entry.grid(row=row, column=col*2+1, padx=8, pady=30, sticky="ew")

# 创建输出参数外框
output_outer_frame = ttk.Frame(predict_root, padding="0 0 0 0", style="TFrame")
output_outer_frame.grid(row=1, column=2, sticky="nsew", padx=(5, 10))

# 设置输出参数外框标题
output_title_label = ttk.Label(output_outer_frame, text="Output panel", font=('Times New Roman', 18, "bold"))
output_title_label.pack(fill="x", padx=140, pady=10)

# 创建输出参数框架
output_frame = ttk.Frame(output_outer_frame, padding="6 6 6 6", width=180)
output_frame.pack(fill="both", expand=True, padx=5, pady=5)

# 调整输出框架的宽度
output_outer_frame.columnconfigure(0, weight=4)
output_outer_frame.columnconfigure(1, weight=1)

# 输出标签和输入框（一列，一行）
output_labels = ['CS']
output_vars = []

# 创建输出标签和输入框（一列，一行）
for i, label in enumerate(output_labels):
    full_label = f"{label}{output_units[label]}"
    ttk.Label(output_frame, text=full_label).grid(row=i, column=0, padx=50, pady=100, sticky="ew")
    
    entry = create_styled_entry(output_frame, width=12)
    entry.grid(row=i, column=1, padx=20, pady=80, sticky="ew")
    output_vars.append(entry)

control_panel_frame = ttk.Frame(predict_root, padding="0 0 0 0", style="TFrame")
control_panel_frame.grid(row=2, column=0, columnspan=3, sticky="ew", padx=10, pady=5)

control_panel_title_label = ttk.Label(control_panel_frame, text="Control panel", font=('Times New Roman', 18, "bold"))
control_panel_title_label.pack(fill="x", padx=590, pady=10)

control_panel_inner_frame = ttk.Frame(control_panel_frame, padding="0 0 0 0")
control_panel_inner_frame.pack(fill="x", padx=5, pady=5)

button_predict = ttk.Button(control_panel_inner_frame, text="Predict", command=update_predict, width=18)
button_predict.pack(side="left", padx=(55, 12), pady=20, anchor="n")

button_clear = ttk.Button(control_panel_inner_frame, text="Clear", command=clear_entries, width=18)
button_clear.pack(side="left", padx=(55, 12), pady=20, anchor="n")

button_save = ttk.Button(control_panel_inner_frame, text="Save", command=save_entries, width=18)
button_save.pack(side="left", padx=(55, 12), pady=20, anchor="n")

button_main_menu = ttk.Button(control_panel_inner_frame, text="Exit", command=return_main_menu, width=18)
button_main_menu.pack(side="left", padx=(55, 12), pady=20, anchor="n")

def check_entry_style():
    test_entry = ttk.Entry(predict_root)
    print("TEntry样式选项:", style.configure("TEntry"))
    test_entry.destroy()
    
    test_tk_entry = tk.Entry(predict_root, font=('Times New Roman', 20, 'bold'))
    print("tk.Entry字体:", test_tk_entry.cget('font'))
    test_tk_entry.destroy()

check_entry_style()

predict_root.mainloop()

可用主题: ('winnative', 'clam', 'alt', 'default', 'classic', 'vista', 'xpnative')
TEntry样式选项: {'foreground': 'black', 'font': '{Times New Roman} 20 bold', 'padding': [5], 'relief': 'sunken'}
tk.Entry字体: {Times New Roman} 20 bold
